In [1]:

import pandas as pd
from datetime import datetime
from utils.paths import Paths

from performance.tools import (
    DailyReturnGenerator,
    ReturnMetricsRunner,
    MetricsInteractiveViewer,
    ReturnDataFrameBuilder,
)
from models.machine_learning import DatasetLoader

## 実際のトレード履歴をリターン化

In [2]:
trade_history = pd.read_csv(Paths.TRADE_HISTORY_CSV)
drg = DailyReturnGenerator(date_series=trade_history["日付"],
                     acquisition_price_series=trade_history["取得価格"],
                     settlement_price_series=trade_history["決済価格"],
                     long_or_short_series=trade_history["売or買"],
                     short_keyphrase="売")
daily_return_df = drg.get()
daily_return_df.tail(2)

,Return
Date,
2025-06-25,-0.002846
2025-06-26,-0.000405


In [3]:
rmr = ReturnMetricsRunner(date_series=daily_return_df.index,
                          return_series=daily_return_df['Return'])
df_dict = rmr.calculate()
miv = MetricsInteractiveViewer(df_dict)
miv.display()

## モデルのリターンデータ読み込み

In [4]:

en48 = f'{Paths.ML_DATASETS_FOLDER}/48sectors_Ensembled_learned_in_250615'
#en48_dataset = DatasetLoader(dataset_root=en48).load_datasets()

In [5]:
lasso54 = f'{Paths.ML_DATASETS_FOLDER}/54sectors_LASSO_learned_in_250623'
trade_secor_numbers = 2
top_slope = 1
eval_start_date = datetime(2022, 1, 1)
eval_end_date = datetime.today()

# ここのゾーンは独立したクラスとして作成
# データセットからリターン算出用データフレームを作成
lasso54_pred_df = ReturnDataFrameBuilder(dataset_root=lasso54, start_date=eval_start_date, end_date=eval_end_date).build()


# これ以降の部分は既存のReturnMetricsRunnerに追加（correction_label_seriesがNoneではないときのみ）
# TargetとPredにRank列を追加
'''
lasso54_for_evaluating_performance['TargetRank'] = lasso54_for_evaluating_performance.groupby('Date')['Target'].rank(ascending=False)
lasso54_for_evaluating_performance['PredRank'] = lasso54_for_evaluating_performance.groupby('Date')['Pred'].rank(ascending=False)

sector_numbers = len(lasso54_for_evaluating_performance.index.get_level_values('Sector').unique())
lasso54_long = lasso54_for_evaluating_performance[lasso54_for_evaluating_performance['PredRank'] <= trade_secor_numbers]
lasso54_short = lasso54_for_evaluating_performance[lasso54_for_evaluating_performance['PredRank'] > sector_numbers - trade_secor_numbers]

if trade_secor_numbers > 1:
    lasso54_long.loc[lasso54_long['PredRank'] == 1, 'Target'] *= top_slope
    lasso54_long.loc[lasso54_long['PredRank'] != 1, 'Target'] *= 1 - (top_slope - 1) / (trade_secor_numbers - 1)
    lasso54_short.loc[lasso54_short['PredRank'] == sector_numbers, 'Target'] *= top_slope
    lasso54_short.loc[lasso54_short['PredRank'] != sector_numbers, 'Target'] *= 1 - (top_slope - 1) / (trade_secor_numbers - 1)
lasso54_long = lasso54_long.groupby('Date')[['Target']].mean()
lasso54_short = - lasso54_short.groupby('Date')[['Target']].mean()
lasso54_return = (lasso54_long + lasso54_short) / 2

lasso54_return
'''


"\nlasso54_for_evaluating_performance['TargetRank'] = lasso54_for_evaluating_performance.groupby('Date')['Target'].rank(ascending=False)\nlasso54_for_evaluating_performance['PredRank'] = lasso54_for_evaluating_performance.groupby('Date')['Pred'].rank(ascending=False)\n\nsector_numbers = len(lasso54_for_evaluating_performance.index.get_level_values('Sector').unique())\nlasso54_long = lasso54_for_evaluating_performance[lasso54_for_evaluating_performance['PredRank'] <= trade_secor_numbers]\nlasso54_short = lasso54_for_evaluating_performance[lasso54_for_evaluating_performance['PredRank'] > sector_numbers - trade_secor_numbers]\n\nif trade_secor_numbers > 1:\n    lasso54_long.loc[lasso54_long['PredRank'] == 1, 'Target'] *= top_slope\n    lasso54_long.loc[lasso54_long['PredRank'] != 1, 'Target'] *= 1 - (top_slope - 1) / (trade_secor_numbers - 1)\n    lasso54_short.loc[lasso54_short['PredRank'] == sector_numbers, 'Target'] *= top_slope\n    lasso54_short.loc[lasso54_short['PredRank'] != secto